In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

In [4]:
import pandas as pd
from datetime import datetime

autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

nome = 'CAFI'
indisponivel = 'indisponivel'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

In [5]:

def abrir_suap():
    
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    #nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    nav.set_page_load_timeout(60)
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    
    # inserindo usuário
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [6]:
# extraindo para lista
def table_to_list():
    tabela = []

    lista = nav.find_element(By.TAG_NAME, 'table')
    td = lista.find_elements(By.TAG_NAME, 'td')

    for i in td:
        tabela.append(i.text)

    return tabela

In [7]:
def build_df(lista):
    df_tabela = pd.DataFrame()


    tombos = []
    ele_des = []
    descricao = []
    cargas = []
    c_contabil = []
    salas = []
    dt_entrada = []
    fornecedores = []
    valores = []

    #  remove o primeiro elemento ta tabela, informação quebrada    
    for excluir in range(0, len(lista), 1201):
        lista.pop(excluir)

    for tombo in range(1, len(lista)+1, 12):
        if lista[tombo] == '':
            lista.pop(tombo)

    # listas para montar o data frame
    for tombo in range(1, len(lista), 12):
        tombos.append(lista[tombo])

    for ed in range(2, len(lista), 12):
        ele_des.append(lista[ed])

    for des in range(3, len(lista), 12):
        descricao.append(lista[des])

    for carga in range(4, len(lista), 12):
        cargas.append(lista[carga])

    for c in range(5, len(lista), 12):
        c_contabil.append(lista[c])

    for sala in range(6, len(lista), 12):
        salas.append(lista[sala])

    for data in range(8, len(lista), 12):
        dt_entrada.append(lista[data])
    
    for f in range(10, len(lista), 12):
        fornecedores.append(lista[f])

    for valor in range(11, len(lista), 12):
        valores.append(lista[valor])



    #  montagem do data frame
    df_tabela['tombo'] = tombos
    df_tabela['tombo'] = df_tabela['tombo'].astype(str).str[:-6]

    df_tabela['ed'] = ele_des

    df_tabela['descricao'] = descricao

    df_tabela['carga'] = cargas
    df_tabela['sala'] = salas

    df_tabela['c_contabil'] = c_contabil

    df_tabela['dt_entrada'] = dt_entrada
#     df_tabela['dt_entrada'] = pd.to_datetime(df_tabela['dt_entrada'], infer_datetime_format=True)

    df_tabela['fornecedor'] = fornecedores
    
    df_tabela['valor'] = valores
    df_tabela['valor'] = df_tabela['valor'].str.replace('.','',regex=True)
    df_tabela['valor'] = df_tabela['valor'].str.replace(',','.',regex=True)
    df_tabela['valor'] = pd.to_numeric(df_tabela['valor'])
    df_tabela['valor'] = df_tabela['valor'].round(2)

    return df_tabela

In [8]:
def pagination():
    
    abrir_suap()
    print(f'Extração iniciada as: {datetime.now()}')
    pag = 1
    campi = 12
    saida = False
    complete_df = pd.DataFrame()
    
    
    while saida is False:
    # cafi->12x  crec->4x    cbja->7x   cbar->8x   ccar->11x    ccsa->13x    cgar->10x   cabl->22x 
    # cipj->6x   cjbg->14x   coli->20x  cpes->5x   cplt->21x    cpmr->18    cvsa->9x    reipe->3x
        try:
            nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
            celula = nav.find_element(By.CLASS_NAME, 'row1')

            if celula:
                suap_list = table_to_list()
                suap_df = build_df(suap_list)
                complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                pag += 1

            else:
                saida = True

        except NoSuchElementException:
            
            try:
                error = nav.find_element(By.ID, 'main-frame-error')
                print(f'Caiu as: {datetime.now()} - na pagina {pag}')

                if error:

                    nav.quit()
                    abrir_suap()
                    print(f'Retomou as: {datetime.now()} na pagina {pag}')
                    nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
                    celula = nav.find_element(By.CLASS_NAME, 'row1')

                    if celula:
                        pag += 1
                        suap_list = table_to_list()
                        suap_df = build_df(suap_list)
                        complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                        print(f'pagina pos erro - {pag}')
                        continue

                    else:
                        break
                        
            except NoSuchElementException:
#                 print('acabaram as paginas')
                break
            
                

    print(f'Extração finalizada as: {datetime.now()} na pagina {pag}')
    print(complete_df.info())
    display(complete_df)
    complete_df.to_excel(f'extracoes_suap/{data_e_hora}_{nome}_relatorio_patrimonio.xlsx', index=False)
    
#     nav.quit()
    

In [9]:
def complement_item(item_list):
    
    abrir_suap()
    
    status = []
    conservation = []
    empenho = []
    ug_ne = []
    uasg = []
    nf = []
    dt_nf = []
    type_entry = []
    supplier = []
    process = []
    linha = 0
    
    print(f'Iniciou as: {datetime.now()}')
    
#     for linha in item_list:
    while linha is not len(item_list):
        
#         print(linha)
        
        try:        
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{item_list[linha]}/")

            st = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[2]/td[2]").get_attribute('innerText')
            status.append(st)

            con = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td[2]").get_attribute('innerText')
            conservation.append(con)

            nav.find_element(By.LINK_TEXT, "Detalhes da Entrada").click()

            try:
                ne = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
                ug = ne[18:22]
                ne = ne[:12]
                empenho.append(ne)
                ug_ne.append(ug)
            # print(empenho)
            except NoSuchElementException as ex:
                empenho.append(indisponivel)
                ug_ne.append(indisponivel)

            try:
                gest = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
                uasg.append(gest)
            except NoSuchElementException as ex:
                uasg.append(indisponivel)
                # print(uasg)

            try:
                note = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
                nf.append(note)
            except NoSuchElementException as ex:
                nf.append(indisponivel)
            # print(nf)

            try:
                data = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
                dt_nf.append(data)
            except NoSuchElementException as ex:
                dt_nf.append(indisponivel)
            # print(data)

            try:
                entry = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[3]/td").get_attribute('innerText')
                type_entry.append(entry)
            except NoSuchElementException as ex:
                type_entry.append(indisponivel)

            try:
                sp = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td/a").get_attribute('innerText')
                supplier.append(sp)
            except NoSuchElementException as ex:
                supplier.append(indisponivel)

            try:
                pc = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[13]/td").get_attribute('innerText')
                process.append(pc)
            except NoSuchElementException as ex:
                process.append(indisponivel)

        except TimeoutException:
            # print(indisponivel)
            empenho.append(indisponivel)
            ug_ne.append(indisponivel)
            uasg.append(indisponivel)
            nf.append(indisponivel)
            dt_nf.append(indisponivel)
            type_entry.append(indisponivel)
            supplier.append(indisponivel)
            process.append(indisponivel)
            print(f'TimeoutException as: {datetime.now()} no item {item_list[linha]}')

            linha += 1
            continue
        
        except NoSuchElementException as ex:
            
            try:
                error = nav.find_element(By.ID, 'id_username')
                print(f'Caiu as: {datetime.now()} - no item {item_list[linha]}')

                if error:

                    nav.quit()
                    abrir_suap()
                    nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{item_list[linha]}/")
                    print(f'Retomou as: {datetime.now()} no item {item_list[linha]}')
                    continue
            
            except NoSuchElementException as ex:
                print('acabaram as paginas')
                break
            
        linha += 1
        continue

            
    table = pd.DataFrame()
    table['status'] = status
    table['consetvation'] = conservation
    table['empenho'] = empenho
    table['ug_empenho'] = ug_ne
    table['uasg'] = uasg
    table['nt_fiscal'] = nf
    table['dt_nf'] = dt_nf
    table['tipo_entrada'] = type_entry
    table['fornecedor'] = supplier
    table['processo'] = process
    print(f'Terminou as: {datetime.now()}')
    print(table.info())
    
    table.to_excel(f'{data_e_hora}_{nome}.xlsx', index=False)
        

#     nav.quit()

In [10]:
pagination()

[WDM] - Downloading: 16.9kB [00:00, 3.91MB/s]                   
[WDM] - Downloading: 16.9kB [00:00, 3.78MB/s]                   
[WDM] - Downloading: 100%|██████████| 2.92M/2.92M [00:00<00:00, 11.6MB/s]
/tmp/ipykernel_25484/892323078.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())


Extração iniciada as: 2023-03-16 10:09:59.253009
Extração finalizada as: 2023-03-16 10:48:47.026065 na pagina 103
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10176 entries, 0 to 10175
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tombo       10176 non-null  object 
 1   ed          10176 non-null  object 
 2   descricao   10176 non-null  object 
 3   carga       10176 non-null  object 
 4   sala        10176 non-null  object 
 5   c_contabil  10176 non-null  object 
 6   dt_entrada  10176 non-null  object 
 7   fornecedor  10176 non-null  object 
 8   valor       10176 non-null  float64
dtypes: float64(1), object(8)
memory usage: 715.6+ KB
None


,tombo,ed,descricao,carga,sala,c_contabil,dt_entrada,fornecedor,valor
0,33,449052-42,182CREC - BANCO P/DESENHO (FIXO COM 75CM DE AL...,VANESSA RAMOS ALVES (CAFI AFI-DEN),,RECIFE,16/06/2010,IFPE - CAMPUS RECIFE (10.767.239/0003-07),0.01
1,3842,449052-42,14471CREC - ARMARIO (DE A O PARA ACESSORIOS DE...,Carlos Eduardo Cabral Rodrigues (CAFI AFI-CCBEC),,RECIFE,16/06/2010,IFPE - CAMPUS RECIFE (10.767.239/0003-07),0.01
2,16809,449052-10,51132CREC - TABELA DE BASQUETEBOL MOVEL (PAR) ...,Ezenildo Emanuel de Lima (CAFI AFI-CODAGR),,AFOGADOS DA INGAZEIRA,16/06/2010,IFPE - CAMPUS RECIFE (10.767.239/0003-07),12698.00
3,17274,449052-35,51615CREC - MICROCOMPUTADOR PESSOAL (NOTEBOOK ...,Ezenildo Emanuel de Lima (CAFI AFI-CODAGR),ANEXO DO ALMOXARIFADO - BLOCO C: LABORATÓRIOS ...,RECIFE,16/06/2010,IFPE - CAMPUS RECIFE (10.767.239/0003-07),4553.00
4,27262,449052-52,"43983CPES - VEICULO (FORD,FIESTA1.6 04P.5PAS.A...",Ezenildo Emanuel de Lima (CAFI AFI-CODAGR),,AFOGADOS DA INGAZEIRA,16/06/2010,IFPE - CAMPUS RECIFE (10.767.239/0003-07),28397.40
...,...,...,...,...,...,...,...,...,...
10171,273932,449052-04,BASTÃO TELESCOPICO,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,AFOGADOS DA INGAZEIRA,07/02/2023,GEOCENTER COMERCIO DE EQUIPAMENTOS OPTICOS (07...,610.00
10172,273933,449052-04,TRIPE DE ALUMINIO,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,AFOGADOS DA INGAZEIRA,07/02/2023,GEOCENTER COMERCIO DE EQUIPAMENTOS OPTICOS (07...,689.00
10173,273934,449052-04,ESTACAO TOTAL TOPOGRAFICA ACOMPANHA ESTOJO DE ...,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,AFOGADOS DA INGAZEIRA,07/02/2023,GEOCENTER COMERCIO DE EQUIPAMENTOS OPTICOS (07...,18471.00
10174,273935,449052-04,NIVEL OPTICO AUTOMATICO ZOOM 24X,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,AFOGADOS DA INGAZEIRA,07/02/2023,GEOCENTER COMERCIO DE EQUIPAMENTOS OPTICOS (07...,900.00
